In [1]:
import pandas as pd
import numpy as np
import spacy
from py2neo import Graph, Node, Relationship


In [2]:
df = pd.read_csv("./data/GraphData.csv")

In [16]:
# Use percentage or a specified number of rows
sample_percentage = 0.001
num_rows = 100


# sampled_df = df.sample(frac=sample_percentage, random_state=42) 
sampled_df = df.sample(n=num_rows, random_state=42)
print(sampled_df.head(5))

sampled_df.to_csv("./data/GraphData_sampled.csv", index=False)



        Subreddit     Word1 Dependency     Word2  \
6193596   Anxiety   knowing       dobj       day   
1973406       BPD   boosted       dobj  finances   
5840048   Anxiety    taking       dobj     carbs   
6722662   Anxiety  universe      nsubj     death   
2766017       BPD     heard      nsubj      help   

                                                  MHlabels  
6193596  {'SYMPTOMS': ['anxiety', 'panic attack'], 'SLE...  
1973406  {'ANXIETY DISORDERS': ['anxiety'], 'DEPRESSIVE...  
5840048  {'SYMPTOMS': ['anxiety'], 'ANXIETY DISORDERS':...  
6722662                                                 {}  
2766017                            {'SYMPTOMS': ['anger']}  


In [27]:

# Create a connection to your Neo4j instance
graph = Graph(uri="bolt://localhost:7687", auth=("neo4j", "letmein1234"))

graph.delete_all()
# Dictionary to store existing nodes
existing_nodes = {}

tx = graph.begin()

# Iterate through the DataFrame and create nodes and relationships
for row in sampled_df.itertuples():

    # Check if the Subreddit node already exists
    existing_subreddit = existing_nodes.get(row.Subreddit)

    if existing_subreddit:
        # If the node exists, use it
        subreddit_node = existing_subreddit
    else:
        # If not, create a new node
        subreddit_node = Node("Subreddit", name=row.Subreddit)
        tx.create(subreddit_node)
        existing_nodes[row.Subreddit] = subreddit_node

    
    # Check if the Word node already exists
    existing_word = existing_nodes.get(row.Word2)    
    if existing_word:
        # If the node exists, use it
        word2_node = existing_word
    else:
         # If not, create a new node
        word2_node = Node("Word", name=row.Word2)
        tx.create(word2_node)
        existing_nodes[row.Word2] = word2_node

    # Use Word1 as the relationship type
    relationship = Relationship(subreddit_node, row.Word1, word2_node)

    # Always create relationships
    tx.create(relationship)

graph.commit(tx)


In [28]:
query = "MATCH (n:Subreddit) RETURN n LIMIT 10"
graph.run(query).data()

# Query to return all nodes in the graph
# query = "MATCH (n) return n"
# graph.run(query).data()

[{'n': Node('Subreddit', name='depression')},
 {'n': Node('Subreddit', name='schizophrenia')},
 {'n': Node('Subreddit', name='mentalillness')},
 {'n': Node('Subreddit', name='bipolar')},
 {'n': Node('Subreddit', name='Anxiety')},
 {'n': Node('Subreddit', name='BPD')}]